In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_excel
from mymachinelearning import ANN
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [2]:
trueSea = np.array(Image.open(r"input files/heightReal.png","r").convert('L'))
#unfixedSea = np.array(Image.open(r"input files/provincesSea.png","r").convert('L'))
#unfixedSea = np.array(Image.open(r"input files/heightSea.png","r").convert('L'))
unfixedSea = np.array(Image.open(r"input files/flatSea.png","r").convert('L'))
#provinceMap = Image.open(r"input files/provinces.png","r").convert('RGB')
#heightMap = Image.open(r"input files/heightmap.png","r")
flatMap = Image.open(r"input files/flatmap.png","r").convert('RGB')

In [3]:
maxpixelx = 16384
maxpixely = 6144

def inner_loop(y0, pmap):
    temp = []
    for x0 in range(maxpixelx):
        #pbar.set_postfix_str(f'x0: {x0}')
        truebool = trueSea[y0,x0]
        if truebool==unfixedSea[y0,x0]:
            temp.append(pmap.getpixel((x0,y0)))
            continue

        #Floodfill through provinceSea until the color is equal to heightbool, save x,y and recolor the province map
        swarm = [(x0,y0)]
        visited = np.zeros((maxpixely,maxpixelx))
        while len(swarm)!=0:
            x, y = swarm[0]

            xt, yt = x, max(y - 1,0)
            if unfixedSea[yt,xt]==truebool: break
            if visited[yt,xt]==0: 
                visited[yt,xt] = 1
                swarm.append((xt,yt))

            xt, yt = max(x - 1,0), y
            if unfixedSea[yt,xt]==truebool: break
            if visited[yt,xt]==0: 
                visited[yt,xt] = 1
                swarm.append((xt,yt))
            
            xt, yt = x, min(y + 1,maxpixely-1)
            if unfixedSea[yt,xt]==truebool: break
            if visited[yt,xt]==0: 
                visited[yt,xt] = 1
                swarm.append((xt,yt))

            xt, yt = min(x + 1,maxpixelx-1), y
            if unfixedSea[yt,xt]==truebool: break
            if visited[yt,xt]==0: 
                visited[yt,xt] = 1
                swarm.append((xt,yt))

            swarm.pop(0)
        temp.append(pmap.getpixel((xt,yt)))
    return temp

In [4]:
result = Parallel(n_jobs=-2)(delayed(inner_loop)(y0, flatMap) for y0 in tqdm(range(maxpixely)))

for y0, temp in enumerate(result):
    for x0, value in enumerate(temp):
        flatMap.putpixel((x0,y0),value)

#provinceMap.save('output files/provinces.png')
#heightMap.save('output files/heightmap.png')
flatMap.save('output files/flatmap.png')

  0%|          | 0/6144 [00:00<?, ?it/s]

# test

In [28]:
heightMap = np.array(Image.open(r"input files/heightmap.png","r").convert('L'))
trueSea = np.array(Image.open(r"input files/heightReal.png","r").convert('L'))

In [29]:
maxpixelx = heightMap.shape[1]
maxpixely = heightMap.shape[0]
#superiorVal = 24
#setVal = 18
coast = 33
seacoast = 30

def inner_loop(y0, heightMap, maxpixelx, maxpixely, superiorVal, setVal):
    temp = []
    for x0 in range(maxpixelx):
        color = heightMap[y0,x0]
        landsea = trueSea[y0,x0]

        if landsea==0: #Has to be sea
            if color>seacoast:
                temp.append(seacoast)
            else: temp.append(color)
        else: #Has to be land
            if color<coast:
                temp.append(coast)
            else: temp.append(color)

        # conditional = True
        # for i in range(1):
        #     xt, yt = x0, max(y0 - 1,0)
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = max(x0 - 1,0), y0
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = x0, min(y0 + 1,maxpixely-1)
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = min(x0 + 1,maxpixelx-1), y0
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = max(x0 - 1,0), max(y0 - 1,0)
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = max(x0 - 1,0), min(y0 + 1,maxpixely-1)
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = min(x0 + 1,maxpixelx-1), min(y0 + 1,maxpixely-1)
        #     if heightMap[yt,xt]==superiorVal: break

        #     xt, yt = min(x0 + 1,maxpixelx-1), max(y0 - 1,0)
        #     if heightMap[yt,xt]==superiorVal: break

        #     conditional = False

        # if conditional==False:
        #     temp.append(color)
        # else: 
        #     temp.append(setVal)
    return temp

result = Parallel(n_jobs=-2)(delayed(inner_loop)(y0, heightMap, maxpixelx, maxpixely, superiorVal, setVal) for y0 in tqdm(range(maxpixely)))

for y0, temp in enumerate(result):
    for x0, value in enumerate(temp):
        heightMap[y0,x0] = value

  0%|          | 0/6144 [00:00<?, ?it/s]

In [30]:
temp = Image.fromarray(heightMap)
temp.save('output files/heightmap.png')

In [ ]:
# riverprovinces = list(range(15077,16005+1))
# bprov = (50,50,50)

# Image.MAX_IMAGE_PIXELS = None
# provimg = Image.open(r'temp.png')
# provimg.convert('RGB')

# data = np.array(provimg)   # "data" is a height x width x 4 numpy array
# r, g, b = data.T # Temporarily unpack the bands for readability

# interest_areas = (r == bprov[0]) & (g == bprov[1]) & (b == bprov[2])

# for ids in riverprovinces:
#     print(ids)
#     color = prov[ids]
#     interest_areas = (r == color[0]) & (g == color[1]) & (b == color[2]) | interest_areas

# data[..., :][interest_areas.T] = bprov # Transpose back needed

# provimg = Image.fromarray(data)
# provimg.save('tempwithoutrivers.png')